In [1]:
import pandas as pd
from datetime import datetime
import openpyxl
from openpyxl import load_workbook
import os

In [6]:
df = pd.read_excel("sharqia.xlsx", sheet_name='python')
df.head()

,DAY,IRRAD,TEMP,TMIN,TMAX,VAP,WIND,RAIN,SNOWDEPTH
0,2000-10-01,17580000.0,23.48,16.99,32.00,14.787259,3.09,0.0,-999
1,2000-10-02,19920000.0,23.96,16.00,33.37,14.585035,2.61,0.0,-999
2,2000-10-03,15890000.0,24.98,18.52,33.17,17.502695,2.81,0.0,-999
3,2000-10-04,18550000.0,26.39,18.78,35.62,18.551383,2.14,0.0,-999
4,2000-10-05,17820000.0,26.20,18.62,35.52,17.268205,1.88,0.0,-999


In [7]:
def to_yyddd(date):
    year = date.year % 100
    doy = date.timetuple().tm_yday
    return f"{year:02d}{doy:03d}"

df["DATE"] = df["DAY"].apply(to_yyddd)

In [8]:
df["SRAD"] = df["IRRAD"] / 1e6      # DSSAT requires MJ/m2/day
df["TMAX"] = df["TMAX"]
df["TMIN"] = df["TMIN"]
df["RAIN"] = df["RAIN"]
df["WIND"] = df["WIND"]              # already m/s or convert if needed

df = df[['DATE', "TMIN", "TMAX", "WIND", "RAIN", "SRAD"]]
df.head()

,DATE,TMIN,TMAX,WIND,RAIN,SRAD
0,00275,16.99,32.00,3.09,0.0,17.58
1,00276,16.00,33.37,2.61,0.0,19.92
2,00277,18.52,33.17,2.81,0.0,15.89
3,00278,18.78,35.62,2.14,0.0,18.55
4,00279,18.62,35.52,1.88,0.0,17.82


# 5. Save as DSSAT-fixed format text

In [17]:
# -----------------------
st = 'EGSL'
station_code = st
station_name = st

for year in range(2001,2005):
    y = f"{year%100 :02d}"  
    
    with open(f"{st}{y}01.WTH", "w") as f:
        # Header
        f.write(f"*WEATHER DATA : {station_name}\n")
        f.write("@ INSI      LAT     LONG  ELEV   TAV   AMP\n")
        f.write(f"  {station_code}    30.50   31.60  20    -99   -99\n")
        f.write("@DATE  SRAD  TMAX  TMIN  RAIN\n")

        # Daily data
        df_y = df[df['DATE'].astype(str).str.startswith(y)]
        for _, row in df_y.iterrows():
            f.write(
                f"{row['DATE']:5s}"
                f"{row['SRAD']:6.2f}"
                f"{row['TMAX']:6.1f}"
                f"{row['TMIN']:6.1f}"
                f"{row['RAIN']:6.1f}\n"
            )
